In [ ]:
#@title 🔑 OpenRouter API Key Provisioning Tool { display-mode: "form" }
#@markdown ### Welcome! This tool helps you create OpenRouter API keys for multiple users.
#@markdown
#@markdown **📋 What you need:**
#@markdown - A provisioning API key from [OpenRouter](https://openrouter.ai/settings/provisioning-keys) (store it in Colab Secrets for security)
#@markdown - A CSV file with user data (must have a **Name** column; **Email**, **Surname**, and **Budget** are optional)
#@markdown
#@markdown **🚀 How to use:**
#@markdown 1. Click the **▶ Run cell** button (or press Shift+Enter)
#@markdown 2. Wait for the interface to load below
#@markdown 3. Follow the 4 steps in the interface that appears
#@markdown 4. Download your results automatically!
#@markdown
#@markdown **🔐 Security Tip:** Set up your provisioning key in Colab Secrets (🔑 icon in left sidebar) with the name `OPENROUTER_PROVISIONING_KEY`
#@markdown
#@markdown ---

# Install and import required libraries
import sys

# Install dependencies
try:
    import pandas as pd
except ImportError:
    !pip install -q pandas
    import pandas as pd

try:
    import ipywidgets as widgets
except ImportError:
    !pip install -q ipywidgets
    import ipywidgets as widgets

try:
    from tqdm import tqdm
except ImportError:
    !pip install -q tqdm
    from tqdm import tqdm

# Standard library imports
import os
import io
import time
import requests
from datetime import datetime
from pathlib import Path
from IPython.display import display, HTML, clear_output

# Check if running in Google Colab
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

print("✅ All libraries loaded successfully\n")

# ============================================================================
# CORE FUNCTIONS
# ============================================================================

def provision_single_key(provision_key, name, label, limit):
    """Create a single OpenRouter API key using the provisioning API."""
    BASE_URL = "https://openrouter.ai/api/v1/keys"
    
    try:
        response = requests.post(
            url=BASE_URL,
            headers={
                "Authorization": f"Bearer {provision_key}",
                "Content-Type": "application/json"
            },
            json={
                "name": name,
                "label": label,
                "limit": limit
            },
            timeout=30
        )
        
        if response.status_code == 201:
            return response.json(), None
        else:
            return None, f"HTTP {response.status_code}: {response.text}"
    except Exception as e:
        return None, str(e)


def provision_keys_bulk(df, provision_key, default_budget):
    """Provision OpenRouter API keys for all users in the DataFrame."""
    results = {
        'successful': 0,
        'failed': 0,
        'errors': []
    }
    
    # Prepare output DataFrame
    output_df = df.copy()
    output_df['api_key'] = None
    
    # Ensure budget column exists
    if 'Budget' not in output_df.columns:
        output_df['Budget'] = default_budget
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Process each row
    for index, row in tqdm(output_df.iterrows(), total=len(output_df), desc="Creating keys"):
        # Get user name
        name_parts = [str(row['Name'])]
        if 'Surname' in row and pd.notna(row['Surname']):
            name_parts.append(str(row['Surname']))
        full_name = "_".join(name_parts)
        
        # Get email if available
        email = str(row['Email']) if 'Email' in row and pd.notna(row['Email']) else full_name
        
        # Determine budget
        budget = default_budget
        if 'Budget' in row and pd.notna(row['Budget']):
            try:
                user_budget = float(row['Budget'])
                if user_budget > 0:
                    budget = user_budget
            except (ValueError, TypeError):
                pass  # Use default budget
        
        # Update budget in output
        output_df.at[index, 'Budget'] = budget
        
        # Create unique identifiers
        key_name = f"APIKey_{timestamp}_{full_name}"
        key_label = f"{timestamp}_{email}"
        
        # Provision the key
        key_data, error = provision_single_key(provision_key, key_name, key_label, budget)
        
        if key_data and 'key' in key_data:
            output_df.at[index, 'api_key'] = key_data['key']
            results['successful'] += 1
        else:
            results['failed'] += 1
            results['errors'].append({
                'name': full_name,
                'error': error or 'Unknown error'
            })
        
        # Rate limiting
        time.sleep(0.2)
    
    return output_df, results


def save_and_download_csv(df, filename):
    """Save CSV to Colab filesystem and trigger download."""
    # Save to /content/
    filepath = f"/content/{filename}"
    df.to_csv(filepath, index=False)
    
    # Download if in Colab
    if IN_COLAB:
        files.download(filepath)
    
    return filepath


# ============================================================================
# GUI CREATION
# ============================================================================

# Check for provisioning key in Colab secrets
provisioning_key_from_secrets = None
if IN_COLAB:
    try:
        from google.colab import userdata
        provisioning_key_from_secrets = userdata.get('OPENROUTER_PROVISIONING_KEY')
    except Exception:
        pass

# Create widgets
if provisioning_key_from_secrets:
    provision_key_widget = widgets.Text(
        value='',
        placeholder='✅ PROVISIONING KEY LOADED - GOOD TO GO',
        description='Provision Key:',
        disabled=True,
        style={'description_width': '120px'},
        layout=widgets.Layout(width='600px')
    )
else:
    provision_key_widget = widgets.Password(
        value='',
        placeholder='⚠️ ENTER YOUR OWN PROVISIONING KEY',
        description='Provision Key:',
        style={'description_width': '120px'},
        layout=widgets.Layout(width='600px')
    )

upload_button = widgets.FileUpload(
    accept='.csv',
    multiple=False,
    description='Upload CSV',
    button_style='info',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='600px')
)

csv_info = widgets.HTML(
    value='<p style="margin-left: 130px; color: #666; font-size: 0.9em;">'
          'CSV should have: <b>Name</b> (required), <b>Email</b>, <b>Surname</b>, <b>Budget</b> (optional)</p>'
)

budget_widget = widgets.BoundedFloatText(
    value=1.00,
    min=0.01,
    max=10000.00,
    step=1.00,
    description='Default Budget:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='300px')
)

budget_info = widgets.HTML(
    value='<span style="margin-left: 10px; color: #666; font-size: 0.9em;">USD per key</span>'
)

create_button = widgets.Button(
    description='🔑 CREATE KEYS',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px'),
    style={'font_weight': 'bold'}
)

output_area = widgets.Output()

# State variables
uploaded_df = None

# ============================================================================
# EVENT HANDLERS
# ============================================================================

def on_upload_change(change):
    """Handle CSV file upload."""
    global uploaded_df
    
    with output_area:
        clear_output()
        
        if len(upload_button.value) == 0:
            return
        
        try:
            # Get uploaded file
            uploaded_file = list(upload_button.value.values())[0]
            content = uploaded_file['content']
            
            # Read CSV
            uploaded_df = pd.read_csv(io.BytesIO(content))
            
            # Validate required columns
            if 'Name' not in uploaded_df.columns:
                print("❌ ERROR: CSV must contain 'Name' column")
                print(f"Found columns: {list(uploaded_df.columns)}")
                uploaded_df = None
                return
            
            print(f"✅ CSV loaded successfully: {len(uploaded_df)} rows")
            print(f"\nColumns found: {list(uploaded_df.columns)}")
            print(f"\n📋 Preview (first 5 rows):")
            display(uploaded_df.head())
            
            # Check for budget column
            if 'Budget' in uploaded_df.columns:
                valid_budgets = uploaded_df['Budget'].apply(lambda x: pd.notna(x) and isinstance(x, (int, float)) and x > 0).sum()
                print(f"\n💰 Budget column found: {valid_budgets}/{len(uploaded_df)} rows have valid budgets")
                print("   Rows without valid budgets will use the default budget")
            else:
                print("\n💰 No Budget column found - all keys will use the default budget")
                
        except Exception as e:
            print(f"❌ Error loading CSV: {e}")
            uploaded_df = None


def on_create_click(button):
    """Handle CREATE KEYS button click."""
    global uploaded_df
    
    with output_area:
        clear_output()
        
        # Validate inputs
        provision_key = provisioning_key_from_secrets or provision_key_widget.value
        
        if not provision_key or provision_key.strip() == '':
            print("❌ ERROR: Provisioning key is required")
            print("Please enter your provisioning key or set it up in Colab Secrets")
            return
        
        if uploaded_df is None:
            print("❌ ERROR: Please upload a CSV file first")
            return
        
        default_budget = budget_widget.value
        
        # Confirmation
        print("🚀 PROVISIONING CONFIGURATION")
        print("="*60)
        print(f"Users to provision: {len(uploaded_df)}")
        print(f"Default budget: ${default_budget:.2f}")
        
        # Calculate total budget
        if 'Budget' in uploaded_df.columns:
            total = 0
            for _, row in uploaded_df.iterrows():
                try:
                    b = float(row['Budget']) if pd.notna(row['Budget']) and float(row['Budget']) > 0 else default_budget
                    total += b
                except (ValueError, TypeError):
                    total += default_budget
            print(f"Estimated total budget: ${total:.2f}")
        else:
            print(f"Estimated total budget: ${default_budget * len(uploaded_df):.2f}")
        
        print("="*60)
        print("\n⏳ Starting key provisioning...\n")
        
        # Provision keys
        try:
            result_df, results = provision_keys_bulk(uploaded_df, provision_key, default_budget)
            
            # Display statistics
            print("\n" + "="*60)
            print("📊 PROVISIONING RESULTS")
            print("="*60)
            print(f"✅ Successful: {results['successful']} / {len(uploaded_df)}")
            print(f"❌ Failed: {results['failed']} / {len(uploaded_df)}")
            
            # Calculate actual total budget
            actual_total = result_df[result_df['api_key'].notna()]['Budget'].sum()
            print(f"💰 Total budget allocated: ${actual_total:.2f}")
            print("="*60)
            
            # Show errors if any
            if results['errors']:
                print("\n⚠️ FAILED KEY CREATIONS:")
                for i, err in enumerate(results['errors'], 1):
                    print(f"{i}. {err['name']}: {err['error']}")
            
            # Save and download results
            if results['successful'] > 0:
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"provisioned_keys_{timestamp}.csv"
                
                print(f"\n💾 Saving results...")
                filepath = save_and_download_csv(result_df, filename)
                
                print(f"✅ Results saved to: {filepath}")
                if IN_COLAB:
                    print(f"📥 Download started automatically")
                
                print(f"\n📋 Preview of results (first 5 rows):")
                display(result_df[['Name', 'Budget', 'api_key']].head())
                
                print("\n⚠️ SECURITY REMINDER:")
                print("• Keep the CSV file with API keys secure")
                print("• Distribute keys through encrypted channels only")
                print("• Never commit API keys to public repositories")
            else:
                print("\n❌ No keys were created successfully. Please check the errors above.")
                
        except Exception as e:
            print(f"\n❌ CRITICAL ERROR: {e}")
            import traceback
            print("\nFull error trace:")
            print(traceback.format_exc())


# Attach event handlers
upload_button.observe(on_upload_change, names='value')
create_button.on_click(on_create_click)

# ============================================================================
# DISPLAY GUI
# ============================================================================

print("🎨 OpenRouter API Key Provisioning Interface\n")

display(widgets.VBox([
    widgets.HTML("<h3>🔑 Step 1: Provisioning Key</h3>"),
    provision_key_widget,
    widgets.HTML("<br>"),
    widgets.HTML("<h3>📁 Step 2: Upload CSV</h3>"),
    upload_button,
    csv_info,
    widgets.HTML("<br>"),
    widgets.HTML("<h3>💰 Step 3: Set Default Budget</h3>"),
    widgets.HBox([budget_widget, budget_info]),
    widgets.HTML("<br>"),
    widgets.HTML("<h3>🚀 Step 4: Create Keys</h3>"),
    create_button,
    widgets.HTML("<br><hr>"),
    output_area
]))